# VacationPy

In [1]:
import pandas as pd
import gmaps
import numpy as np
import json 
from citipy import citipy
import requests
import matplotlib.pyplot as plt

from api_keys import gkey

### Getting the data

In [2]:
cities_df = pd.read_csv("../WeatherPy/Weather_per_city.csv")
cities_df.head()

,Lat,Long,City,Country,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,52.146062,14.819762,eisenhuttenstadt,de,NaN,NaN,NaN,NaN,NaN
1,-22.970000,-42.030000,arraial do cabo,br,71.60,71.60,88.0,92.0,10.29
2,-15.820000,-50.610000,itapirapua,br,69.31,69.31,74.0,14.0,3.18
3,38.320000,26.760000,urla,tr,56.91,59.00,87.0,25.0,4.70
4,-6.100000,-39.450000,acopiara,br,71.83,71.83,93.0,52.0,2.08


### Nww DataFrame with fitting weather criteria

In [3]:
#Dropping rows with null values
cities_df.dropna(inplace=True)

# Narrowing down the consitions to find the cities with ideal weather 
cities_df = cities_df[cities_df["Wind Speed (mph)"] < 10]
cities_df = cities_df[cities_df["Cloudiness (%)"] == 0]
cities_df = cities_df[cities_df["Temperature (F)"] < 80]
cities_df = cities_df[cities_df["Temperature (F)"] > 70]

# Saving the coordinates of the found cities
locations = cities_df[["Lat", "Long"]]

# Creating the humidity dataframe to use as weight in the map
humidity = cities_df["Humidity (%)"]

### Hotel map 

In [97]:
# Creating the array of coordinates to use when findind the city
city_coords = []
for i in range(len(cities_df)):
    temp = list(cities_df['Lat'])[i]
    temp2 = list(cities_df['Long'])[i]
    var = str(temp)+", "+str(temp2)
    city_coords.append(var)

In [98]:
# Finding the nearests hotels within a radius of 5000 m
target_type = "lodging"
radius = 5000

gmaps.configure(api_key=gkey)

In [110]:
# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

latit = []
longit = []
name = []
region = []

params = {
    "location": city_coords[0],
    "radius": radius,
    "type": target_type,
    "key": gkey
}
    
response = requests.get(base_url, params = params).json() 
    
latit.append(response["results"][0]["geometry"]["location"]["lat"])
longit.append(response["results"][0]["geometry"]["location"]["lng"])
name.append(response["results"][0]["name"]) 
region.append(response["results"][0]["vicinity"])

# Storing the values in a dataframe
hotel = {
    "Lat": latit, 
    "Long": longit,
    "Name": name,
    "Region": region  
}

hotel_df = pd.DataFrame(hotel, columns = ["Lat", "Long", "Name", "Region"], index = [0])

In [108]:
# Dataframe of found hotels
hotel_df

,Lat,Long,Name,Region
0,29.032724,21.545805,Jalu Hotel,Jalu


In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [102]:
# Generating the map
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(
    locations,
    weights = humidity,
    dissipating = False,
    max_intensity = max(cities_df["Humidity (%)"]),
    point_radius = 5
)

# Puttign the pins in the maps 
marker_locations =  hotel_df[["Lat", "Long"]]
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))